# 필요한 라이브러리 불러오기
* 실습을 위한 pytorch라이브러리를 불러옵니다.

In [1]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

/Users/isang-yun/PycharmProjects/AiPaper_samplecode/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 생성자 및 판별자 모델 정의

In [3]:
latent_dim = 100

''' 생성자 (Generator) 클래스 정의 '''
class Generator(nn.Module):
    def __int__(self):
        super(Generator, self).__int__()

hello world!
